In [112]:
# generate info about mobile sites from mobile ID for both SANBS and Vitalant
import pandas as pd
import numpy as np
import datetime as dt
from datetime import timedelta

import matplotlib.pyplot as plt
import seaborn as sns

df=pd.read_csv("../1_data/private/SANBSdata.csv", error_bad_lines=False)
#df=pd.read_csv("../1_data/private/VitalantData.csv")

/tmp/ipykernel_1529881/280045915.py:10: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df=pd.read_csv("../1_data/private/SANBSdata.csv", error_bad_lines=False)
/tmp/ipykernel_1529881/280045915.py:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("../1_data/private/SANBSdata.csv", error_bad_lines=False)


In [113]:
## for SANBS

# look at mobile ID to see the number of times a year it is visited 
#filter unique dates - some go for 2 or 3 consectutive days
df.drop(df[df['Visit_Date'].isna()].index, inplace=True) #remove any missing visit dates

df["Visit_Date"] = df["Visit_Date"].astype("datetime64") #convert to date time object
df.drop(df[(df['Fixed']== 'Lapsed Clinic') | (df['Fixed']=='Processing Lab')| (df['Fixed']=='Unallocated') 
           | (df["Fixed"]== 'Laboratorium') |(df["Fixed"]=='Doctor')].index, inplace=True)

df['Fixed_mobile'] = np.where(df['Fixed'] == 'Mobile Clinic', 0, 1) #mobile is 0, fixed is 1

df['Year']= df['Visit_Date'].dt.strftime('%Y')
df['Year']=df.Year.astype('int32')


In [114]:
df=df.loc[df.Fixed_mobile==0] #keep mobile sites
df_year_id=df.groupby(['Year', 'MobileID'])["Visit_Date"].apply(list)
df_year_id


Year  MobileID
2015  ELE001      [2015-04-21 00:00:00, 2015-02-10 00:00:00, 201...
      ELE005      [2015-05-13 00:00:00, 2015-05-13 00:00:00, 201...
      ELE008      [2015-05-13 00:00:00, 2015-09-02 00:00:00, 201...
      ELE013      [2015-07-29 00:00:00, 2015-05-13 00:00:00, 201...
      ELE014      [2015-03-09 00:00:00, 2015-10-12 00:00:00, 201...
                                        ...                        
2022  WRE791      [2022-09-02 00:00:00, 2022-09-02 00:00:00, 202...
      WRE792      [2022-11-18 00:00:00, 2022-11-18 00:00:00, 202...
      WRE793      [2022-11-15 00:00:00, 2022-11-15 00:00:00, 202...
      WRE794      [2022-11-02 00:00:00, 2022-11-02 00:00:00, 202...
      WRE795      [2022-11-23 00:00:00, 2022-11-23 00:00:00, 202...
Name: Visit_Date, Length: 39133, dtype: object

In [120]:
df_mID = df_year_id.to_frame()
df_mID["Visit_Date"]=df_mID['Visit_Date'].apply(lambda x: np.unique(x))
df_mID["Visit_Date"]=df_mID['Visit_Date'].apply(lambda x: sorted(x))
df_mID

Visit_Date
Year MobileID                                                   
2015 ELE001    [2015-02-10 00:00:00, 2015-04-21 00:00:00, 201...
     ELE005    [2015-03-12 00:00:00, 2015-05-13 00:00:00, 201...
     ELE008    [2015-02-25 00:00:00, 2015-05-13 00:00:00, 201...
     ELE013    [2015-02-18 00:00:00, 2015-05-13 00:00:00, 201...
     ELE014    [2015-03-09 00:00:00, 2015-05-11 00:00:00, 201...
...                                                          ...
2022 WRE791           [2022-09-02 00:00:00, 2022-11-04 00:00:00]
     WRE792           [2022-09-02 00:00:00, 2022-11-18 00:00:00]
     WRE793           [2022-09-20 00:00:00, 2022-11-15 00:00:00]
     WRE794                                [2022-11-02 00:00:00]
     WRE795    [2022-11-22 00:00:00, 2022-11-23 00:00:00, 202...

[39133 rows x 1 columns]

In [121]:
# iterate through rows keep unique date that do not occur with one or two day of each other to determine frequency of drive
def filter_dates(date_list):
    filtered_dates = [date_list[0]]
    last_date = pd.to_datetime(date_list[0])

    for date_str in date_list[1:]:
        date = pd.to_datetime(date_str)
        duration = date - last_date
        days = duration.days
        if days >= 2:
            filtered_dates.append(date_str)
        last_date = date

    return filtered_dates

df_mID['Visit_Dates'] = df_mID['Visit_Date'].apply(filter_dates)


In [122]:
df_mID["Counts"]=df_mID['Visit_Dates'].apply(lambda x: len(x))


In [123]:
df_mID.loc[(2022,'EEE078')].head(25)

Visit_Date     [2022-01-02 00:00:00, 2022-01-03 00:00:00, 202...
Visit_Dates    [2022-01-02 00:00:00, 2022-01-10 00:00:00, 202...
Counts                                                        35
Name: (2022, EEE078), dtype: object

In [126]:
df_1= df_mID.explode('Visit_Date')
df_1.sort_values(by=['MobileID', 'Visit_Date'], axis = 0, inplace=True)
df_1=df_1.droplevel('Year')


In [133]:
df_2= df_mID.explode('Visit_Dates')
df_2.sort_values(by=['MobileID', 'Visit_Dates'], axis = 0, inplace=True)
df_2=df_2.droplevel('Year')
df_2.reset_index(inplace=True)


In [134]:
def count_id_in_next_12_months(row):
    mask = (df_2['MobileID'] == row['MobileID']) & (df_2['Visit_Dates'] > row['Visit_Date']) & (df_2['Visit_Dates'] <= row['next_year'])
    return mask.sum()

df_1.reset_index(inplace=True)
df_1['next_year'] = df_1['Visit_Date'] + pd.DateOffset(months=12)
df_1['count_in_next_12_months'] = df_1.apply(count_id_in_next_12_months, axis=1)


In [135]:
df_1.loc[df_1.MobileID=='EEE078'].head(50)

,index,MobileID,Visit_Date,Visit_Dates,Counts,next_year,count_in_next_12_months
1962,1962,EEE078,2017-01-03,"[2017-01-03 00:00:00, 2017-01-09 00:00:00, 201...",44,2018-01-03,44
1963,1963,EEE078,2017-01-04,"[2017-01-03 00:00:00, 2017-01-09 00:00:00, 201...",44,2018-01-04,44
1964,1964,EEE078,2017-01-05,"[2017-01-03 00:00:00, 2017-01-09 00:00:00, 201...",44,2018-01-05,44
1965,1965,EEE078,2017-01-06,"[2017-01-03 00:00:00, 2017-01-09 00:00:00, 201...",44,2018-01-06,44
1966,1966,EEE078,2017-01-07,"[2017-01-03 00:00:00, 2017-01-09 00:00:00, 201...",44,2018-01-07,44
1967,1967,EEE078,2017-01-09,"[2017-01-03 00:00:00, 2017-01-09 00:00:00, 201...",44,2018-01-09,44
1968,1968,EEE078,2017-01-11,"[2017-01-03 00:00:00, 2017-01-09 00:00:00, 201...",44,2018-01-11,44
1969,1969,EEE078,2017-01-12,"[2017-01-03 00:00:00, 2017-01-09 00:00:00, 201...",44,2018-01-12,44
1970,1970,EEE078,2017-01-13,"[2017-01-03 00:00:00, 2017-01-09 00:00:00, 201...",44,2018-01-13,44
1971,1971,EEE078,2017-01-14,"[2017-01-03 00:00:00, 2017-01-09 00:00:00, 201...",44,2018-01-14,44


In [137]:
df_1.to_csv(("../1_data/private/mID_table.csv"), index=False)


In [138]:
# For Vitalant
df=pd.read_csv("../1_data/private/VitalantData.csv")

/tmp/ipykernel_1529881/86117606.py:2: DtypeWarning: Columns (2,13,23,25,28,38,40,41,49,50,51,52,53,54,55,56,57,58,59,61,62,63) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("../1_data/private/VitalantData.csv")


In [139]:
df['MobileID']=df['COLLECTION_SITE_KEY']
df["Visit_Date"] = pd.to_datetime(df['DON_DATE_KEY'], format='%Y%m%d') #convert to datetime object

In [140]:

# look at mobile ID to see the number of times a year it is visited 
#filter unique dates - some go for 2 or 3 consectutive days
df.drop(df[df['Visit_Date'].isna()].index, inplace=True) #remove any missing visit dates
df["Visit_Date"] = df["Visit_Date"].astype("datetime64") #convert to date time object

df['Fixed_mobile'] = np.where(df["DRIVE_TYPE_FIXED_MOBILE"] == 'Fixed', 1, 0) #mobile is 0, fixed is 1

df['Year']= df['Visit_Date'].dt.strftime('%Y')
df['Year']=df.Year.astype('int32')

df=df.loc[df.Fixed_mobile==0] #keep mobile sites
df_year_id=df.groupby(['Year', 'MobileID'])["Visit_Date"].apply(list)
df_year_id


/tmp/ipykernel_1529881/790932034.py:4: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  df["Visit_Date"] = df["Visit_Date"].astype("datetime64") #convert to date time object


Year  MobileID
2017  8.0         [2017-09-13 00:00:00, 2017-02-26 00:00:00, 201...
      9.0         [2017-12-03 00:00:00, 2017-11-14 00:00:00, 201...
      10.0        [2017-10-23 00:00:00, 2017-08-31 00:00:00, 201...
      11.0        [2017-02-21 00:00:00, 2017-01-31 00:00:00, 201...
      12.0        [2017-06-25 00:00:00, 2017-10-15 00:00:00, 201...
                                        ...                        
2022  89342.0     [2022-10-14 00:00:00, 2022-10-04 00:00:00, 202...
      121238.0    [2022-04-11 00:00:00, 2022-02-08 00:00:00, 202...
      121250.0    [2022-10-17 00:00:00, 2022-02-18 00:00:00, 202...
      121264.0    [2022-02-07 00:00:00, 2022-10-06 00:00:00, 202...
      121272.0    [2022-02-01 00:00:00, 2022-06-24 00:00:00, 202...
Name: Visit_Date, Length: 1503, dtype: object

In [141]:
df_mID = df_year_id.to_frame()
df_mID["Visit_Date"]=df_mID['Visit_Date'].apply(lambda x: np.unique(x))
df_mID["Visit_Date"]=df_mID['Visit_Date'].apply(lambda x: sorted(x))
df_mID

Visit_Date
Year MobileID                                                   
2017 8.0       [2017-01-04 00:00:00, 2017-01-05 00:00:00, 201...
     9.0       [2017-01-05 00:00:00, 2017-01-06 00:00:00, 201...
     10.0      [2017-01-05 00:00:00, 2017-01-10 00:00:00, 201...
     11.0      [2017-01-05 00:00:00, 2017-01-06 00:00:00, 201...
     12.0      [2017-01-09 00:00:00, 2017-01-10 00:00:00, 201...
...                                                          ...
2022 89342.0   [2022-10-03 00:00:00, 2022-10-04 00:00:00, 202...
     121238.0  [2022-01-05 00:00:00, 2022-01-06 00:00:00, 202...
     121250.0  [2022-01-02 00:00:00, 2022-01-03 00:00:00, 202...
     121264.0  [2022-01-02 00:00:00, 2022-01-03 00:00:00, 202...
     121272.0  [2022-01-03 00:00:00, 2022-01-04 00:00:00, 202...

[1503 rows x 1 columns]

In [142]:
df_mID['Visit_Dates'] = df_mID['Visit_Date'].apply(filter_dates)
df_mID["Counts"]=df_mID['Visit_Dates'].apply(lambda x: len(x))

df_1= df_mID.explode('Visit_Date')
df_1.sort_values(by=['MobileID', 'Visit_Date'], axis = 0, inplace=True)
df_1=df_1.droplevel('Year')

df_2= df_mID.explode('Visit_Dates')
df_2.sort_values(by=['MobileID', 'Visit_Dates'], axis = 0, inplace=True)
df_2=df_2.droplevel('Year')
df_2.reset_index(inplace=True)

df_1.reset_index(inplace=True)
df_1['next_year'] = df_1['Visit_Date'] + pd.DateOffset(months=12)
df_1['count_in_next_12_months'] = df_1.apply(count_id_in_next_12_months, axis=1)



In [143]:
df_1.to_csv(("../1_data/private/mID_table_V.csv"), index=False)